In [90]:
import numpy as np
import pandas as pd
import scipy.constants as sc
from pygmid import Lookup as lk

#### Technology data

In [91]:
n = lk('../../techsweep/simulation/nfet_01v8_lvt.mat')
p = lk('../../techsweep/simulation/pfet_01v8.mat')

#### Specifications

In [92]:
T=300; A0 =50; PM=87; NOI=110e-6

#### Design choices

In [93]:
# n input pair, p mirror
type = ['n', 'n', 'p']

# pick channel lengths for gain requirement
l = np.array([1.0, 0.5,  1.0])

# pick large gm/ID for input pairs, smaller gm/ID for mirror (to reduce noise)
gm_id = np.zeros(len(type))
gm_id[1] = 20
gm2_gm1 = 0.5
gm_id[2] = gm2_gm1*gm_id[1]
gm_id[0] = gm_id[1]

#### Sizing and benchmarking

In [94]:
# calculate load capacitance for noise spec
# NOI = ( gamma[1]*(1+2*gamma[2]/gamma[1]) * sc.Boltzmann*T/cltot)**0.5
gamma = np.zeros(len(type))
gamma[1] = n.lookup('STH_GM', GM_ID=gm_id[1], L=l[1])/(4*sc.Boltzmann*T)
gamma[2] = n.lookup('STH_GM', GM_ID=gm_id[2], L=l[2])/(4*sc.Boltzmann*T)
cl = gamma[1]*(1+2*gamma[2]/gamma[1]) * sc.Boltzmann*T/NOI**2

# calculate mirror pole frequency and UGF that guarantees phase margin (with some margin)
# phi = np.arctan(UGF/fp2)*180/np.pi
fp2 = 0.5*p.lookup('GM_CGG', GM_ID=gm_id[2], L=l[2])/2/np.pi
phi = 90-PM
fp2_UGF = 1/np.tan(phi/180*np.pi)
UGF = fp2/fp2_UGF

# calculate gm1 based on UGF target
# UGF = gm[1]/cltot/2/np.pi
gm = np.zeros(len(type))
gm[1] = UGF*cl*2*np.pi

# calculate current and all widths
ib = 2*gm[1]/gm_id[1]
id = ib*np.array([1, 0.5, 0.5])
w = np.zeros(len(type))
w[0] = id[0]/n.lookup('ID_W', GM_ID=gm_id[0], L=l[0])
w[1] = id[1]/n.lookup('ID_W', GM_ID=gm_id[1], L=l[1])
w[2] = id[2]/n.lookup('ID_W', GM_ID=gm_id[2], L=l[2])
wfing = 5
nf = 1+np.floor_divide(w, wfing)

df = pd.DataFrame( [cl/1e-12, UGF/1e6, ib/1e-6], \
                   ['cl (pF)', 'UGF (MHz)', 'ib (uA)'], columns=['Value']); df.round(2)

,Value
cl (pF),1.78
UGF (MHz),9.43
ib (uA),10.55


In [95]:
df = pd.DataFrame( [w, l, nf], \
                   ['w (um)', 'l (um)', 'nf'], columns=['M0', 'M1', 'M2']); df.round(2)

,M0,M1,M2
w (um),23.23,5.83,1.13
l (um),1.00,0.50,1.00
nf,5.00,2.00,1.00


#### Write spice include file

In [99]:
# override with baseline values if desired
if False:
  ib = 10e-6; cl = 1e-12
  l = np.array([1.0, 0.5,  1.0])
  w = np.array([2.0, 20.0, 2.0])
  nf = np.array([2, 4, 2])

with open('sizing_ota-5t.spice', 'w') as file:
    file.write(".param ib = " + "{:.2e}".format(ib) + '\n')
    file.write(".param cl = " + "{:.2e}".format(cl) + '\n')
    for i in range(len(l)):
      file.write(".param w%d = " % i + "{:.2f}".format(w[i]) + '\n')
      file.write(".param l%d = " % i + "{:.2f}".format(l[i]) + '\n')
      file.write(".param nf%d = " % i + "{:.2f}".format(nf[i]) + '\n')